In [ ]:
 # Dependencies and Setup
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore

In [ ]:
# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [ ]:
# Read School and Student Data File and store into Pandas Data Frames
school_dat = pd.read_csv(school_data_to_load)
student_dat = pd.read_csv(student_data_to_load) 

In [ ]:
# Flawed dataset, should not move forward until resolved.  
# Concerns in terms of number of student records and qualifications/titles included in the student name field: 
    #229 DDS, 141 DVM, 347 MD, 82 PhD, 193 Dr. unspecified, 224 Mr., 132 Mrs., 25 Ms. The last three could be valid, but 
    # still suspicious for their low occurence rates. 
# But what the heck, I'm proceeding anyway.      

In [ ]:
#  Prep the School level data

In [ ]:
#  Compute the budget per student and add to school dataset
school_dat['budget_per_stu'] = school_dat['budget'] / school_dat['size']

In [ ]:
#  Create per student budget bins and add to school dataset
perstu_bins = [0, 600, 630, 650, 675]
perstu_bin_labels = ["<$600", "$600-629", "$630-649", "$650-675"]
school_dat["$ Per Student"] = pd.cut(school_dat['budget_per_stu'], perstu_bins, labels=perstu_bin_labels)

In [ ]:
#  Create school size bins and add to student dataset
studn_bins = [0, 1000, 2000, 3000, 5000]
studn_bin_labels = ['Under 1,000', '1,000-1,999', '2,000-2,999', 'Over 3,000']
school_dat["Student N"] = pd.cut(school_dat['size'], studn_bins, labels=studn_bin_labels)

In [ ]:
#  Combine the student and school data sets into a single dataset
comb_dat = pd.merge(student_dat, school_dat, how="left", on=["school_name", "school_name"])

In [ ]:
#  Compute pass/fail and overall score 
pass_read = comb_dat['reading_score'] >= 70
pass_math = comb_dat['math_score'] >= 70
over_score = (comb_dat['reading_score'] + comb_dat['math_score']) / 2
comb_dat['over_score'] = over_score
pass_over = comb_dat['over_score'] >= 70

In [ ]:
# Add new variables to the file and converted from True/False to 0/1
comb_dat['pass_read'] = pass_read
comb_dat['pass_math'] = pass_math
comb_dat['pass_over'] = pass_over
comb_dat[['pass_read', 'pass_math', 'pass_over']] = (comb_dat[['pass_read', 'pass_math', 'pass_over']]).astype(int)

In [ ]:
# Typically overall pass/fail for theses types of tests would require passing both tests individually, so I threw this 
# decision rule into mix to see how they compared.  
pass_both = comb_dat['pass_read'] * comb_dat['pass_math']
comb_dat['pass_both'] = pass_both

In [ ]:
# District Summary

In [ ]:
# Brought in and did remaining computations on variables for the District Summary.  Then added column names
ds_tab_dat = pd.DataFrame([[len(school_dat), np.sum(school_dat['size']), np.sum(school_dat['budget']), np.mean(comb_dat['math_score']),
               np.mean(comb_dat['reading_score']), np.mean(comb_dat['pass_math']) * 100, np.mean(comb_dat['pass_read']) * 100,
               np.mean(comb_dat['pass_over']) * 100, np.mean(comb_dat['pass_both']) * 100]])
ds_tab_dat.columns = (['Total Schools','Total Students','Total Budget','Mean Math Score','Mean Reading Score',
                       '% Pass Math','% Pass Reading', '% Pass Overall', "% Pass Reading & Math"])

In [ ]:
# Print District Summary
ds_tab_dat

In [ ]:
# School Summary

In [ ]:
# Brought in data for the School Summary.  Then added column names
school_tab = comb_dat[['school_name', 'size','budget','budget_per_stu','math_score','reading_score','pass_math','pass_read','pass_over','pass_both']]
school_grouped_dat = school_tab.groupby(['school_name'])
school_tab_M = school_grouped_dat.mean()
school_tab_M.columns = (['School Size','Total Budget', '$ per Student','Mean Math Score','Mean Reading Score',
                       '% Pass Math','% Pass Reading', '% Pass Overall', "% Pass Reading & Math"])

In [ ]:
# Print School Summary
school_tab_M

In [ ]:
#  Identify Top Performing Schools and print Top Five based on Overall Pass Rates
school_tab_M = school_tab_M.sort_values('% Pass Overall', ascending=False)
school_tab_M.head(5)

In [ ]:
#  Identify Bottom Performing Schools and print Bottom Five based on Overall Pass Rates
school_tab_M = school_tab_M.sort_values('% Pass Overall', ascending=True)
school_tab_M.head(5)

In [ ]:
#  Math and Reading Scores by Grade Levels
#  Create DataFrames for each grade including both sets of test scores.

grade_9 = comb_dat.loc[comb_dat['grade'] == '9th', ['school_name','grade','math_score', 'reading_score']]
grade_10 = comb_dat.loc[comb_dat['grade'] == '10th', ['school_name','grade','math_score', 'reading_score']]
grade_11 = comb_dat.loc[comb_dat['grade'] == '11th', ['school_name','grade','math_score', 'reading_score']]
grade_12 = comb_dat.loc[comb_dat['grade'] == '12th', ['school_name','grade','math_score', 'reading_score']]


In [ ]:
# Compute means for each of the test score columns in each of the Grade-level specific DataFrames.

grade_9_grouped_dat = grade_9.groupby(['school_name'])
grade_9_M = grade_9_grouped_dat.mean()
grade_10_grouped_dat = grade_10.groupby(['school_name'])
grade_10_M = grade_10_grouped_dat.mean()
grade_11_grouped_dat = grade_11.groupby(['school_name'])
grade_11_M = grade_11_grouped_dat.mean()
grade_12_grouped_dat = grade_12.groupby(['school_name'])
grade_12_M = grade_12_grouped_dat.mean()
grade_9_M.reset_index(inplace=True)
grade_10_M.reset_index(inplace=True)
grade_11_M.reset_index(inplace=True)
grade_12_M.reset_index(inplace=True)

In [ ]:
# assemble columns from data frames to produce Math results by Grade Level

math_by_grade = grade_9_M.drop('reading_score', axis = 1)
math_by_grade.rename(columns = {'math_score' : '9th Grade Math'}, inplace = True)
math_by_grade['10th Grade Math'] = grade_10_M['math_score']
math_by_grade['11th Grade Math'] = grade_11_M['math_score']
math_by_grade['12th Grade Math'] = grade_12_M['math_score']

In [ ]:
math_by_grade

In [ ]:
# assemble columns from data frames to produce Reading results by Grade Level

reading_by_grade = grade_9_M.drop('math_score', axis = 1)
reading_by_grade.rename(columns = {'reading_score' : '9th Grade Reading'}, inplace = True)
reading_by_grade['10th Grade Reading'] = grade_10_M['reading_score']
reading_by_grade['11th Grade Reading'] = grade_11_M['reading_score']
reading_by_grade['12th Grade Reading'] = grade_12_M['reading_score']

In [ ]:
reading_by_grade

In [ ]:
#  Use Bins created earlier to compute scores and pass rates by per student spending. 

spend_dat = comb_dat[['$ Per Student','math_score','reading_score','pass_math','pass_read','pass_over','pass_both']]
spend_grouped_dat = spend_dat.groupby(['$ Per Student'])
spend_tab_M = spend_grouped_dat.mean() 
spend_tab_M.columns = (['Mean Math Score','Mean Reading Score',
                       '% Pass Math','% Pass Reading', '% Pass Overall', "% Pass Reading & Math"])

In [ ]:
spend_tab_M

In [ ]:
# Scores by School Size: Perform the same operations as above, based on school size.

In [ ]:
ssize_dat = comb_dat[['Student N','math_score','reading_score','pass_math','pass_read','pass_over','pass_both']]
ssize_grouped_dat = ssize_dat.groupby(['Student N'])
ssize_tab_M = ssize_grouped_dat.mean()
ssize_tab_M.columns = (['Mean Math Score','Mean Reading Score',
                       '% Pass Math','% Pass Reading', '% Pass Overall', "% Pass Reading & Math"])

In [ ]:
ssize_tab_M

In [ ]:
#  Use existing school types to compute scores and pass rates by per student spending. 

type_dat = comb_dat[['type','math_score','reading_score','pass_math','pass_read','pass_over','pass_both']]
type_grouped_dat = type_dat.groupby(['type'])
type_tab_M = type_grouped_dat.mean()
type_tab_M.columns = (['Mean Math Score','Mean Reading Score',
                       '% Pass Math','% Pass Reading', '% Pass Overall', "% Pass Reading & Math"])

In [ ]:
type_tab_M

In [ ]:
# Conclusions: 
## Data is suspect. 
## The lowest scores in all categories were from distict schools, no Charter School had lower average test scores 
##    than any of the District schools. 
## Charter schools had fewer students than District schools and their per student budgets were generally lower than the 
## District schools.  